# Подбор гиперпараметров LightGBM

## Загрузка модулей

В качестве библиотеки для подбора гиперпараметров мы будем использовать optuna, модель - LightGBM (для совместимости с пайплайном используется обёртка LGBMClassifier), отбор гиперпараметров будет производиться с помощью двух функций оптимизации.

In [1]:
# Модель
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from helpers.data import DataPreprocessor

# Данные
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler

%matplotlib inline

In [2]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

def save_submission(model, subname):
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    preds = model.predict(test_df)
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)

# Загрузка датасетов
train_df = pd.read_csv(TRAIN_DATASET)
train_df = train_df.drop_duplicates()
test_df = pd.read_csv(TEST_DATASET)

# Разбиение на тренировочный и тестовый датасет
X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Нормализация для кв/тестов
X_train = pd.concat([X_train, y_train], axis=1)

X_train = X_train[X_train['Длительность'] < 400]
X_train = X_train[(X_train['Сумма'] > 2) & (X_train['Сумма'] < 10)]
X_train = X_train[X_train['До поставки'] < 300]
X_train = X_train[X_train['Дней между 0_1'] < 400]
X_train = X_train[X_train['Количество изменений после согласований'] < 2000]
X_train = X_train[X_train['Количество'] < 300000]

y_train = X_train['y']
X_train = X_train.drop('y', axis=1)

X_general, y_general = train_df.iloc[:, :-1], train_df.iloc[:, -1]

In [3]:
data_p = DataPreprocessor()
X_ = data_p.fit_transform(X_general, y_general)

In [ ]:
def objective_f1_macro(trial: opt.Trial) -> float:
    """ Функция оптимизации F1 (macro).
    
    Использует библиотеку optuna для подбора параметров из
    заданного диапазона. В качестве оптимизируемой метрики 
    выступает F1 (macro).
    
    Параметры:
        trial: экзмепляр optuna.Trial, представляющей собой
        историю оптимизации целевой функции.
        
    Пример:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=100)
        
    Возвращает F1 (macro) метрику для подобранных параметров."""
    
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1)
    n_estimators = trial.suggest_int('n_estimators', 1300, 3000)
    max_depth = trial.suggest_int('max_depth', 6, 21)
    max_bin = trial.suggest_int('max_bin', 64, 200),
    num_leaves = trial.suggest_int('num_leaves', 32, 260)
    reg_lambda = trial.suggest_float('l2_reg', 0.01, 1)

    # Модель
    data_preprocessor = DataPreprocessor(CAT_FEATURES, SCALE_FEATURES, 
                                         DROP_FEATURES, RENAME_COLS)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        num_leaves=num_leaves,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        force_col_wise=True
    )

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    cv_score = cross_val_score(pipeline, X_general, y_general, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()

    return accuracy